**Code for constructing networks from brain fMRI data  
Author: Jithin K. Sreedharan**

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import scipy.io as sio
import os

# Minimum spanning tree technique

In [ ]:
i_graph_name = 0
cor_mat_folder = 'cortex/data_mat/' # Change it accordingly
output_folder = 'cortex/graphs_spanningtree_180/'
for file_temp in os.listdir(cor_mat_folder):
    file_temp = cor_mat_folder+file_temp
    dict_temp = sio.loadmat(file_temp)
    mat_temp = dict_temp['corr_mat']
    mat_temp = mat_temp[:180,:180]
    mat_temp = np.absolute(mat_temp)

    if mat_temp.shape != (180,180):
        raise ValueError('Dimension of '+file_temp+' is not (180,180)')
    H = nx.Graph()
    # 10-nearest neighbors (higher correlation values) of each node in G are added into H.
    for i in range(180):
        ind_temp = np.argsort(-mat_temp[i,:])[:10]
        edges_temp = [(i,ii) for ii in ind_temp]
        H.add_edges_from(edges_temp)
    L_0 = nx.from_numpy_array(mat_temp)
    L = nx.minimum_spanning_tree(L_0,weight = 'weight')
    # Combine G and H: Union of nodesets and edgesets of G and H
    G = nx.compose(H,L)
    # Relabel the nodes IDs to start from 1 instead of 0. 
    # This is to match the parcel index in Glasser et al. paper to identify the associated brain region
    mapping = {old_label:old_label+1 for old_label in G.nodes()}
    G = nx.relabel_nodes(G, mapping)
    if nx.is_connected(G) == 0:
        print("not connected, something wrong!")
        break     
    graph_file_name = output_folder+'graph_connectome_' + str(i_graph_name)+'.txt'
    nx.write_edgelist(G,graph_file_name,data= False)
    density_graph = 2*G.number_of_edges()/(180*180)
    print("density(G): {0} ".format(density_graph))
    i_graph_name = i_graph_name+1
    H.clear()
    L_0.clear()
    L.clear()
    G.clear()

# Threshold technique

In [ ]:
i = 0
cor_mat_folder = 'cortex/data_mat/' # Change it accordingly
output_folder = 'cortex/graphs_threshold_180/'
for file_temp in os.listdir(cor_mat_folder):
    file_temp = cor_mat_folder+file_temp
    dict_temp = sio.loadmat(file_temp)
    mat_temp = dict_temp['corr_mat']
    mat_temp = mat_temp[:180,:180]
    mat_temp = np.absolute(mat_temp)

    if mat_temp.shape != (180,180):
        raise ValueError('Dimension of '+file_temp+' is not (180,180)')
    
    for perc_temp in np.arange(99,40,-2):
        perc_val = np.percentile(mat_temp,perc_temp)
        temp_mat = (mat_temp > perc_val).astype(int)
        np.fill_diagonal(temp_mat,0)
        G = nx.from_numpy_array(temp_mat)
        if nx.is_connected(G):
            break
    # Relabel the nodes IDs to start from 1 instead of 0. 
    # This is to match the parcel index in Glasser et al. paper to identify the associated brain region
    mapping = {old_label:old_label+1 for old_label in G.nodes()}
    G = nx.relabel_nodes(G, mapping)
    graph_file_name = output_folder + 'graph_connectome_' + str(i)+'.txt'
    nx.write_edgelist(G,graph_file_name,data= False)
    density_graph = 2*G.number_of_edges()/(360*360)
    print("perc_temp: {0}, perc_val: {1}, density(G): {2} ".format(perc_temp,perc_val,density_graph))
    i = i+1